In [1]:
%reload_ext autoreload
%autoreload 2

! [ -L /datasets ] && rm -f /datasets
! ln -s /data/datasets/cv /datasets

## 需掌握知识点

1. 输入层与输出层
- 降维（Flatten）层
- 线性组合（Linear）层

In [1]:
%reset -f

from pyr.app.k12ai import EasyaiClassifier, EasyaiTrainer
import torch
from torch import nn

class CustomClassifier(EasyaiClassifier):

    def prepare_dataset(self):
        """
        chestxray: 2个分类
            0: 'normal' 正常
            1: 'covid', 新冠
        """
        return self.load_chestxray()
    
    def build_model(self):
        class FCNetwork(nn.Module):
            def __init__(self,):
                super().__init__()
                self.fc_net = nn.Sequential()
                self.fc_net.add_module('batchnorm', nn.BatchNorm2d(num_features=3, momentum=0.1))
                self.fc_net.add_module('flatten', nn.Flatten(start_dim=1, end_dim=-1))
                self.fc_net.add_module('linear1', nn.Linear(in_features=150528, out_features=50, bias=True))
                self.fc_net.add_module('linear2', nn.Linear(in_features=50, out_features=2, bias=True))
      
            def forward(self, x):
                return self.fc_net(x)
        return FCNetwork()
    
    def train_dataloader(self):
        return self.get_dataloader(phase='train', input_size=224, batch_size=32)
     
    def training_step(self, batch, batch_idx):
        x, y, _ = batch
        y_hat = self(x)
        loss = self.cross_entropy(y_hat, y, reduction='mean') # 损失方法
        with torch.no_grad():
            accuracy = (torch.argmax(y_hat, axis=1) == y).float().mean()
        return {'loss': loss, 'progress_bar': {'acc': accuracy}}

    def val_dataloader(self):
        return self.get_dataloader(phase='val', input_size=224, batch_size=32)
    
    def validation_step(self, batch, batch_idx):
        x, y, _ = batch
        y_hat = self(x)
        loss = self.cross_entropy(y_hat, y, reduction='mean')
        accuracy = (torch.argmax(y_hat, axis=1) == y).float().mean()
        return {'loss': loss, 'acc': accuracy}
        
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['acc'] for x in outputs]).mean()
        return {'progress_bar': {'val_loss': avg_loss, 'val_acc': avg_acc}}
    
    def test_dataloader(self):
        return self.get_dataloader(phase='test', input_size=224, batch_size=32)
    
    
trainer = EasyaiTrainer(max_epochs=5, model_summary='full')

model = CustomClassifier()

# 训练
trainer.fit(model);

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


--------------------------------------------------------------------------------
{'label_names': ['normal', 'covid'],
 'mean': [0.5262, 0.5255, 0.5282],
 'num_classes': 2,
 'num_records': 402,
 'std': [0.1794, 0.1792, 0.1798]}


RuntimeError: CUDA error: out of memory